In [4]:
# MeCabとIPAdicのインストール
!apt-get -y install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 ipadic

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [5]:
# rouge-scoreライブラリのインストール
!pip install rouge-score

In [6]:
import ipadic
import MeCab
from collections import defaultdict
import pandas as pd
from rouge_score import rouge_scorer, scoring
import os
from typing import Union, List

# pandasの小数点以下の桁数を3に設定する
pd.options.display.precision = 3

# --- 1. ROUGE計算のためのヘルパー関数 ---

def convert_words_to_ids(
    predictions: List[str], references: List[str]
) -> tuple[List[str], List[str]]:
    """単語列をID列に変換"""
    # 単語にユニークなIDを割り当てるためのdefaultdictを作成する
    word2id = defaultdict(lambda: len(word2id))

    # 単語区切りの文字列をID文字列に変換する
    pred_ids = [
        " ".join([str(word2id[w]) for w in p.split()])
        for p in predictions
    ]
    ref_ids = [
        " ".join([str(word2id[w]) for w in r.split()])
        for r in references
    ]
    return pred_ids, ref_ids

def compute_rouge(
    predictions: List[str], references: List[str]
) -> dict[str, dict[str, float]]:
    """単語分割済みのリストを受け取りROUGEを算出"""
    # RougeScorerを設定する
    rouge = rouge_scorer.RougeScorer(
        rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=False
    )
    aggregator = scoring.BootstrapAggregator()

    # 単語列をID列に変換する (日本語の正確な評価のため)
    pred_ids, ref_ids = convert_words_to_ids(predictions, references)

    # ROUGEスコアを計算して結果を集約する
    for pred, ref in zip(pred_ids, ref_ids):
        aggregator.add_scores(rouge.score(ref, pred))

    scores = aggregator.aggregate()
    # F値 (fmeasure) の中間値 (mid) を返す
    return {k: v.mid for k, v in scores.items()}

# --- 2. ファイル/文字列 入力処理関数 ---

def load_data(input_data: Union[str, List[str]], tagger: MeCab.Tagger) -> List[str]:
    """
    入力データ（ファイルパスまたは文字列のリスト）を読み込み、単語分割して返す
    """
    data_list = []

    if isinstance(input_data, str):
        # 文字列がファイルパスとして存在する場合
        if os.path.exists(input_data):
            print(f"✅ ファイル '{input_data}' からデータを読み込みます。")
            with open(input_data, 'r', encoding='utf-8') as f:
                # ファイルの各行を1つの文として処理
                data_list = [line.strip() for line in f if line.strip()]
        else:
            # 存在しない場合は、その文字列を単一のデータとして扱う
            print(f"💡 入力文字列を単一のデータとして扱います。")
            data_list = [input_data]

    elif isinstance(input_data, list):
        # リストが与えられた場合
        print(f"💡 リスト入力されたデータを扱います。")
        data_list = input_data

    # MeCabで単語分割（分かち書き）を行う
    wakati_list = [tagger.parse(text).strip() for text in data_list]
    return wakati_list

def compute_rouge_from_input(
    predictions: Union[str, List[str]],
    references: Union[str, List[str]]
) -> pd.DataFrame:
    """
    生成文と正解文の入力形式（ファイルパスまたはリスト）を処理し、ROUGEスコアを算出する
    """
    # IPAdicを用いたMeCabを設定（分かち書きモード）
    tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")

    # データ読み込みと単語分割
    ref_wakati_list = load_data(references, tagger)
    pred_wakati_list = load_data(predictions, tagger)

    if len(ref_wakati_list) != len(pred_wakati_list):
        raise ValueError(
            f"参照文と生成文の数が一致しません。参照文: {len(ref_wakati_list)}、生成文: {len(pred_wakati_list)}"
        )

    # ROUGEスコアの計算
    rouge_scores = compute_rouge(pred_wakati_list, ref_wakati_list)

    # 結果をPandas DataFrameとして整形して表示
    df = pd.DataFrame.from_dict(rouge_scores, orient="index")
    return df

# --- 3. 実行例（データの準備） ---

# ① 直接文字列/リストとして与える例
print("--- 実行例 1: 直接文字列（単一）を与える場合 ---")
single_ref = "「春」をキーワードに星加アナウンサーが松山・石手を歩く。春の河川敷でビーチバレーの特訓をする若者やこの春入学を控える新一年生、自宅の壁一面にレトロ看板を「貼る」夫婦に遭遇。食欲の春！一日７５０本売れる食パンに、老舗うどん店の「メロンうどん」とは？（写真は一場面）"
single_pred = "春の石手寺で、新酒やメロンうどん、ぷるっぷる食パンが楽しめる。ビーチバレーの試合や、杜氏の噂も紹介され、地元の風物詩が紹介される。皆さんの「私にとっての春」を聞かせてください。"
df_single = compute_rouge_from_input(single_pred, single_ref)
print("✅ スコア結果 (単一文字列):")
display(df_single)
print("-" * 50)


# ② 複数の文をリストとして与える例
print("--- 実行例 2: リスト（複数）を与える場合 ---")
multi_refs = ["私は昨日ケーキを食べた。", "明日の天気は晴れです。"]
multi_preds = ["昨日、私はケーキを食べる。", "明日の天気が晴れだ。"]
df_multi = compute_rouge_from_input(multi_preds, multi_refs)
print("✅ スコア結果 (リスト):")
display(df_multi)
print("-" * 50)


# ③ ファイルパスとして与える例 (ファイルが存在しない場合は動作しません)
# 🚨 動作確認のために、以下の2つのファイルを作成してから実行してください。
#     ref.txt:
#     私は昨日ケーキを食べた。
#     明日の天気は晴れです。
#
#     pred.txt:
#     昨日、私はケーキを食べる。
#     明日の天気が晴れだ。

# print("--- 実行例 3: ファイルパスとして与える場合 ---")
# file_ref = "ref.txt"
# file_pred = "pred.txt"
# try:
#     df_file = compute_rouge_from_input(file_pred, file_ref)
#     print("✅ スコア結果 (ファイル):")
#     display(df_file)
# except FileNotFoundError:
#     print("ファイルが存在しません。'ref.txt'と'pred.txt'を作成して試してください。")

--- 実行例 1: 直接文字列（単一）を与える場合 ---
💡 入力文字列を単一のデータとして扱います。
💡 入力文字列を単一のデータとして扱います。
✅ スコア結果 (単一文字列):


,precision,recall,fmeasure
rouge1,0.431,0.265,0.328
rouge2,0.140,0.085,0.106
rougeL,0.235,0.145,0.179


--------------------------------------------------
--- 実行例 2: リスト（複数）を与える場合 ---
💡 リスト入力されたデータを扱います。
💡 リスト入力されたデータを扱います。
✅ スコア結果 (リスト):


,precision,recall,fmeasure
rouge1,0.732,0.732,0.732
rouge2,0.310,0.310,0.310
rougeL,0.670,0.670,0.670


--------------------------------------------------
